In [2]:
import os

In [3]:
%pwd

'c:\\Users\\SACHIN\\PycharmProjects\\Obesity-Risk-\\notebook'

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\SACHIN\\PycharmProjects\\Obesity-Risk-'

In [6]:
import pandas as pd
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,LabelEncoder,MinMaxScaler
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix

C:\Users\SACHIN\AppData\Local\Temp\ipykernel_2772\941171740.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
df = pd.read_csv('artifacts/data_ingestion/train_data.csv')


In [8]:
print(df.head())

   id  Gender        Age    Height      Weight family_history_with_overweight  \
0   0    Male  24.443011  1.699998   81.669950                            yes   
1   1  Female  18.000000  1.560000   57.000000                            yes   
2   2  Female  18.000000  1.711460   50.165754                            yes   
3   3  Female  20.952737  1.710730  131.274851                            yes   
4   4    Male  31.641081  1.914186   93.798055                            yes   

  FAVC      FCVC       NCP        CAEC SMOKE      CH2O SCC       FAF  \
0  yes  2.000000  2.983297   Sometimes    no  2.763573  no  0.000000   
1  yes  2.000000  3.000000  Frequently    no  2.000000  no  1.000000   
2  yes  1.880534  1.411685   Sometimes    no  1.910378  no  0.866045   
3  yes  3.000000  3.000000   Sometimes    no  1.674061  no  1.467863   
4  yes  2.679664  1.971472   Sometimes    no  1.979848  no  1.967973   

        TUE       CALC                 MTRANS           NObeyesdad  
0  0.976473

In [9]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20758 entries, 0 to 20757
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              20758 non-null  int64  
 1   Gender                          20758 non-null  object 
 2   Age                             20758 non-null  float64
 3   Height                          20758 non-null  float64
 4   Weight                          20758 non-null  float64
 5   family_history_with_overweight  20758 non-null  object 
 6   FAVC                            20758 non-null  object 
 7   FCVC                            20758 non-null  float64
 8   NCP                             20758 non-null  float64
 9   CAEC                            20758 non-null  object 
 10  SMOKE                           20758 non-null  object 
 11  CH2O                            20758 non-null  float64
 12  SCC                             

In [10]:
df['BMI'] = df['Weight'] / df['Height'] * df['Height']

bins = [0, 18, 30, 50, np.inf]

labels = ['Teen', 'Young Adult', 'Adult', 'Senior']

df['Age'] = pd.cut(df['Age'], bins=bins, labels=labels)

df = pd.get_dummies(df, columns=['Age'], drop_first=True)

In [11]:
df

,id,Gender,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad,BMI,Age_Young Adult,Age_Adult,Age_Senior
0,0,Male,1.699998,81.669950,yes,yes,2.000000,2.983297,Sometimes,no,...,no,0.000000,0.976473,Sometimes,Public_Transportation,Overweight_Level_II,81.669950,True,False,False
1,1,Female,1.560000,57.000000,yes,yes,2.000000,3.000000,Frequently,no,...,no,1.000000,1.000000,no,Automobile,Normal_Weight,57.000000,False,False,False
2,2,Female,1.711460,50.165754,yes,yes,1.880534,1.411685,Sometimes,no,...,no,0.866045,1.673584,no,Public_Transportation,Insufficient_Weight,50.165754,False,False,False
3,3,Female,1.710730,131.274851,yes,yes,3.000000,3.000000,Sometimes,no,...,no,1.467863,0.780199,Sometimes,Public_Transportation,Obesity_Type_III,131.274851,True,False,False
4,4,Male,1.914186,93.798055,yes,yes,2.679664,1.971472,Sometimes,no,...,no,1.967973,0.931721,Sometimes,Public_Transportation,Overweight_Level_II,93.798055,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20753,20753,Male,1.766626,114.187096,yes,yes,2.919584,3.000000,Sometimes,no,...,no,1.330519,0.196680,Sometimes,Public_Transportation,Obesity_Type_II,114.187096,True,False,False
20754,20754,Male,1.710000,50.000000,no,yes,3.000000,4.000000,Frequently,no,...,no,2.000000,1.000000,Sometimes,Public_Transportation,Insufficient_Weight,50.000000,False,False,False
20755,20755,Male,1.819557,105.580491,yes,yes,2.407817,3.000000,Sometimes,no,...,no,1.158040,1.198439,no,Public_Transportation,Obesity_Type_II,105.580491,True,False,False
20756,20756,Male,1.700000,83.520113,yes,yes,2.671238,1.971472,Sometimes,no,...,no,0.000000,0.973834,no,Automobile,Overweight_Level_II,83.520113,False,True,False


In [12]:
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df['family_history_with_overweight'] = le.fit_transform(df['family_history_with_overweight'])
df['FAVC'] = le.fit_transform(df['FAVC'])
df['CAEC'] = le.fit_transform(df['CAEC'])
df['SMOKE'] = le.fit_transform(df['SMOKE'])
df['SCC'] = le.fit_transform(df['SCC'])
df['CALC'] = le.fit_transform(df['CALC'])
df['MTRANS'] = le.fit_transform(df['MTRANS'])

In [13]:
map_list={0:'Insufficient_Weight', 1:'Normal_Weight', 2:'Obesity_Type_I', 3:'Obesity_Type_II',
 4:'Obesity_Type_III', 5:'Overweight_Level_I' ,6:'Overweight_Level_II'}

remap={'Insufficient_Weight':0 ,'Normal_Weight':1 ,'Obesity_Type_I':2 ,'Obesity_Type_II':3,
 'Obesity_Type_III':4, 'Overweight_Level_I':5 ,'Overweight_Level_II':6}

In [14]:
X = df.drop(['id', 'NObeyesdad'], axis=1)

In [15]:
sc = StandardScaler()
X = sc.fit_transform(X)

In [16]:
print(X)

[[ 1.0041516  -0.00282826 -0.23571264 ...  0.62292115 -0.39184519
  -0.05732898]
 [-0.99586557 -1.60629083 -1.17093118 ... -1.60533962 -0.39184519
  -0.05732898]
 [-0.99586557  0.12845138 -1.43001211 ... -1.60533962 -0.39184519
  -0.05732898]
 ...
 [ 1.0041516   1.36653688  0.6707173  ...  0.62292115 -0.39184519
  -0.05732898]
 [ 1.0041516  -0.00280536 -0.16557441 ... -1.60533962  2.55202828
  -0.05732898]
 [ 1.0041516   1.33206194  1.14664506 ...  0.62292115 -0.39184519
  -0.05732898]]


In [29]:
numeric_features = df.select_dtypes(include = [int, float]).columns.drop(['id'])
categorical_features = df.select_dtypes(include = object).columns

In [31]:
numeric_features

Index(['Gender', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC',
       'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE', 'CALC',
       'MTRANS', 'BMI'],
      dtype='object')

In [18]:
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.3, random_state=42)

In [19]:
def model_evaluation(y_true, predicted):
    ac = accuracy_score(y_true,predicted)
    return ac


In [20]:
models = {
    'XGBClassifier': XGBClassifier(),
    'LogisticRegression': LogisticRegression(),
    'RandomForestClassifier': RandomForestClassifier(),
    'AdaBoostClassifier': AdaBoostClassifier(),
    'DecisionTreeClassifier': DecisionTreeClassifier(),
    'CatBoostClassifier': CatBoostClassifier(),
    'KNeighborsClassifier': KNeighborsClassifier()
}

In [23]:
model_list = []
ac_list = []

In [24]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    pred = model.predict(X_test)

    model_ac = model_evaluation(y_test, pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print(f"accuracy of model{model_ac}")
    ac_list.append(model_ac)

sl = pd.DataFrame(list(zip(model_list, ac_list)), columns=['models', "accuracy_score"])
print(sl)

XGBClassifier
accuracy of model0.8967565831727682


c:\Users\SACHIN\miniconda3\envs\obesity\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression
accuracy of model0.8604688503532434
RandomForestClassifier
accuracy of model0.8962748876043674


c:\Users\SACHIN\miniconda3\envs\obesity\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier
accuracy of model0.43978805394990367
DecisionTreeClassifier
accuracy of model0.8423249839434811
Learning rate set to 0.090698
0:	learn: 1.6572195	total: 271ms	remaining: 4m 30s
1:	learn: 1.4633072	total: 354ms	remaining: 2m 56s
2:	learn: 1.3211834	total: 417ms	remaining: 2m 18s
3:	learn: 1.2066822	total: 510ms	remaining: 2m 7s
4:	learn: 1.1148623	total: 586ms	remaining: 1m 56s
5:	learn: 1.0379090	total: 656ms	remaining: 1m 48s
6:	learn: 0.9716047	total: 779ms	remaining: 1m 50s
7:	learn: 0.9169570	total: 910ms	remaining: 1m 52s
8:	learn: 0.8696716	total: 1.25s	remaining: 2m 17s
9:	learn: 0.8261835	total: 1.53s	remaining: 2m 31s
10:	learn: 0.7899321	total: 1.74s	remaining: 2m 36s
11:	learn: 0.7564847	total: 1.88s	remaining: 2m 35s
12:	learn: 0.7255320	total: 2.06s	remaining: 2m 36s
13:	learn: 0.6971784	total: 2.16s	remaining: 2m 31s
14:	learn: 0.6737185	total: 2.22s	remaining: 2m 25s
15:	learn: 0.6521020	total: 2.31s	remaining: 2m 21s
16:	learn: 0.6294330	total: 2.41s	

In [25]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
pred = xgb.predict(X_test)
print(accuracy_score(y_test, pred))


0.8967565831727682
[4 5 2 ... 0 2 0]


In [26]:
params = {
        'grow_policy': ["depthwise", "lossguide"],
        'n_estimators': stats.randint(50, 1000),
        'learning_rate': stats.uniform(0.01, 1.0),
        'gamma' : stats.uniform( 1e-9, 1.0),
        'subsample': stats.uniform( 0.25, 1.0),
        #'colsample_bytree': stats.uniform( 0.25, 1.0),
        'colsample_bytree': [0, 0.2, 0.5,0.8,1],
        'max_depth': stats.randint(0, 24),
        'min_child_weight': stats.randint( 1, 30),
        'reg_lambda': stats.uniform(1e-9, 10.0),
        'reg_alpha': stats.uniform(1e-9, 10.0),
    }

In [27]:
gcv = RandomizedSearchCV(xgb, params, scoring='accuracy', cv=5, n_iter=10, verbose=3, n_jobs=-1)
gcv.fit(X_train, y_train)
pred = gcv.predict(X_test)
print(gcv.best_params_)
print(accuracy_score(y_test, pred))

Fitting 5 folds for each of 10 candidates, totalling 50 fits


c:\Users\SACHIN\miniconda3\envs\obesity\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\SACHIN\miniconda3\envs\obesity\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\SACHIN\miniconda3\envs\obesity\Lib\site-packages\xgboost\core.py", line 730, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\Users\SACHIN\miniconda3\envs\obesity\Lib\site-packages\xgboost\sklearn.py", line 1519, in fit
    self._Booster = 

{'colsample_bytree': 0.5, 'gamma': 0.541300902169748, 'grow_policy': 'depthwise', 'learning_rate': 0.3673508322530775, 'max_depth': 10, 'min_child_weight': 12, 'n_estimators': 517, 'reg_alpha': 3.352771379658738, 'reg_lambda': 7.775901872135643, 'subsample': 0.6157721490254899}
0.9022157996146436
